In [32]:
import time
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from stargazer.stargazer import Stargazer
from functools import reduce
from IPython.core.display import HTML
import webbrowser
from datetime import datetime
from IPython.display import display, Latex

In [26]:
#Open all correct dataframes
btc_1d = pd.read_csv('BFX-BTCUSD-1d.csv', usecols=['timestamp', 'open', 'close', 'high', 'low', 'volume'])
btc_1h = pd.read_csv('BFX-BTCUSD-1h.csv', usecols=['timestamp', 'open', 'close', 'high', 'low', 'volume'])
btc_15m = pd.read_csv('BFX-BTCUSD-15m.csv', usecols=['timestamp', 'open', 'close', 'high', 'low', 'volume'])
btc_1m = pd.read_csv('BFX-BTCUSD-1m.csv', usecols=['timestamp', 'open', 'close', 'high', 'low', 'volume'])

usdt_1d = pd.read_csv('BFX-USTUSD-1d.csv', usecols=['timestamp', 'open', 'close', 'high', 'low', 'volume'])
usdt_1h = pd.read_csv('BFX-USTUSD-1h.csv', usecols=['timestamp', 'open', 'close', 'high', 'low', 'volume'])
usdt_15m = pd.read_csv('BFX-USTUSD-15m.csv', usecols=['timestamp', 'open', 'close', 'high', 'low', 'volume'])
usdt_1m = pd.read_csv('BFX-USTUSD-1m.csv', usecols=['timestamp', 'open', 'close', 'high', 'low', 'volume'])

usdc_1d = pd.read_csv('BNC-USDCUSDT-1d.csv', usecols=['timestamp', 'open', 'close', 'high', 'low', 'volume'])
usdc_1h = pd.read_csv('BNC-USDCUSDT-1h.csv', usecols=['timestamp', 'open', 'close', 'high', 'low', 'volume'])
usdc_15m = pd.read_csv('BNC-USDCUSDT-15m.csv', usecols=['timestamp', 'open', 'close', 'high', 'low', 'volume'])
usdc_1m = pd.read_csv('BNC-USDCUSDT-1m.csv', usecols=['timestamp', 'open', 'close', 'high', 'low', 'volume'])

busd_1d = pd.read_csv('BNC-BUSDUSDT-1d.csv', usecols=['timestamp', 'open', 'close', 'high', 'low', 'volume'])
busd_1h = pd.read_csv('BNC-BUSDUSDT-1h.csv', usecols=['timestamp', 'open', 'close', 'high', 'low', 'volume'])
busd_15m = pd.read_csv('BNC-BUSDUSDT-15m.csv', usecols=['timestamp', 'open', 'close', 'high', 'low', 'volume'])
busd_1m = pd.read_csv('BNC-BUSDUSDT-1m.csv', usecols=['timestamp', 'open', 'close', 'high', 'low', 'volume'])

dai_1d = pd.read_csv('BNC-USDTDAI-1d.csv', usecols=['timestamp', 'open', 'close', 'high', 'low', 'volume'])
dai_1h = pd.read_csv('BNC-USDTDAI-1h.csv', usecols=['timestamp', 'open', 'close', 'high', 'low', 'volume'])
dai_15m = pd.read_csv('BNC-USDTDAI-15m.csv', usecols=['timestamp', 'open', 'close', 'high', 'low', 'volume'])
dai_1m = pd.read_csv('BNC-USDTDAI-1m.csv', usecols=['timestamp', 'open', 'close', 'high', 'low', 'volume'])

In [39]:
def transform_and_merge(df1, df2, df3, df4, df5, tokenA, tokenB, tokenC, tokenD, tokenE, timeframe):
    
    #use only relevant columns
    df1_copy = df1[['timestamp', 'close', 'volume']].copy()
    # df2 is USDT
    df2_copy = df2[['timestamp', 'close', 'volume']].copy()
    df3_copy = df3[['timestamp', 'close', 'volume']].copy()
    df4_copy = df4[['timestamp', 'close', 'volume']].copy()
    df5_copy = df5[['timestamp', 'close', 'volume']].copy()
    
    # normalize Binance Stablecoins 
    stables = [[df3_copy, tokenC], [df4_copy, tokenD], [df5_copy, tokenE]]
    for stable in stables:
        temp_stable = df2_copy.merge(stable[0], how='left', on=['timestamp'])
        temp_stable['close'] = temp_stable['close_y'] * temp_stable['close_x']
        temp_stable['volume'] = temp_stable['volume_y']
        stable[0] = temp_stable[['timestamp', 'close', 'volume']].copy()
        
    # Calculate log returns for all tokens + rename all columns
    lst = [[df1_copy, tokenA], [df2_copy, tokenB], [df3_copy, tokenC], [df4_copy, tokenD], [df5_copy, tokenE]]
    for pair in lst:
        column_name = 'return_' + pair[1]
        pair[0].columns = ['timestamp', 'close_' + pair[1], 'volume_' + pair[1]]
        pair[0][column_name] = (pair[0]['close_' + pair[1]].pct_change())
        pair[0][column_name + '_log'] = (np.log(1 + pair[0][column_name])) * 100
        
    # Make columns for every quantile relevant for BTC
    df1_copy['Dq_10'] = df1_copy['return_BTC_log'].apply(lambda x: 1 if x < df1_copy['return_BTC_log'].quantile(.1) else 0)
    df1_copy['Dq_05'] = df1_copy['return_BTC_log'].apply(lambda x: 1 if x < df1_copy['return_BTC_log'].quantile(.05) else 0)
    df1_copy['Dq_01'] = df1_copy['return_BTC_log'].apply(lambda x: 1 if x < df1_copy['return_BTC_log'].quantile(.01) else 0)
    
    # Merge all dataframes
    merged = df1_copy.copy()
    lst2 = [df2_copy, df3_copy, df4_copy, df5_copy]
    for df in lst2:
        merged = merged.merge(df, how='left', on=['timestamp'])
    
    merged.set_index('timestamp', inplace=True)
    
    est = smf.ols(formula='return_USDT_log ~  return_BTC_log * Dq_10 + return_BTC_log * Dq_05 + return_BTC_log * Dq_01', data=merged).fit(cov_type='HAC', cov_kwds={'maxlags':1})
    est4 = smf.ols(formula='return_DAI_log ~  return_BTC_log * Dq_10 + return_BTC_log * Dq_05 + return_BTC_log * Dq_01', data=merged).fit(cov_type='HAC', cov_kwds={'maxlags':1})
    est3 = smf.ols(formula='return_BUSD_log ~  return_BTC_log * Dq_10 + return_BTC_log * Dq_05 + return_BTC_log * Dq_01', data=merged).fit(cov_type='HAC', cov_kwds={'maxlags':1})
    est2 = smf.ols(formula='return_USDC_log ~  return_BTC_log * Dq_10 + return_BTC_log * Dq_05 + return_BTC_log * Dq_01', data=merged).fit(cov_type='HAC', cov_kwds={'maxlags':1})

    stargazer = Stargazer([est, est2, est3, est4])

    stargazer.custom_columns(['USDT', 'USDC', 'BUSD', 'DAI'], [1, 1, 1, 1])
    stargazer.show_model_numbers(False)
    stargazer.title(f'Safe Haven Test: Regression results of the {timeframe} timeframe')
        
    return stargazer

In [43]:
# 15-minute timeframe
transform_and_merge(btc_15m, usdt_15m, dai_15m, busd_15m, usdc_15m, "BTC","USDT", "DAI", "BUSD", "USDC", "15-minute")

In [41]:
# 1-hour timeframe
transform_and_merge(btc_1h, usdt_1h, dai_1h, busd_1h, usdc_1h, "BTC","USDT", "DAI", "BUSD", "USDC", "1-hour")

In [ ]:
# Daily timeframe
transform_and_merge(btc_1d, usdt_1d, dai_1d, busd_1d, usdc_1d, "BTC","USDT", "DAI", "BUSD", "USDC", "daily")